In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [32]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [6]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_market_data)

In [7]:
# Create a DataFrame with the scaled data
scaled_df = pd.DataFrame(scaled_data, columns=df_market_data.columns)

# Copy the crypto names from the original DataFrame
scaled_df['coin_id'] = df_market_data.index

# Set the coin_id column as index
scaled_df = scaled_df.set_index('coin_id')

# Display the scaled DataFrame
print(scaled_df.head().to_markdown(index=True, numalign="left", stralign="left"))

| coin_id      | price_change_percentage_24h   | price_change_percentage_7d   | price_change_percentage_14d   | price_change_percentage_30d   | price_change_percentage_60d   | price_change_percentage_200d   | price_change_percentage_1y   |
|:-------------|:------------------------------|:-----------------------------|:------------------------------|:------------------------------|:------------------------------|:-------------------------------|:-----------------------------|
| bitcoin      | 0.508529                      | 0.493193                     | 0.7722                        | 0.23546                       | -0.0674951                    | -0.355953                      | -0.251637                    |
| ethereum     | 0.185446                      | 0.934445                     | 0.558692                      | -0.0543409                    | -0.273483                     | -0.115759                      | -0.199352                    |
| tether       | 0.021774               

---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [12]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

In [14]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans_model = KMeans(n_clusters=k, random_state=42)

    # 2. Fit the model to the data using `df_market_data_scaled`
    kmeans_model.fit(scaled_df)

    # 3. Append the model.inertia_ to the inertia list
    inertia_values.append(kmeans_model.inertia_)

# Display the inertia values for each k
for k, inertia in zip(k_values, inertia_values):
    print(f"k = {k}, Inertia = {inertia}")

k = 1, Inertia = 287.0
k = 2, Inertia = 212.1233420748626
k = 3, Inertia = 170.19357087970297
k = 4, Inertia = 79.02243535120975
k = 5, Inertia = 71.0979135124442
k = 6, Inertia = 66.1088267804475
k = 7, Inertia = 54.48812315624117
k = 8, Inertia = 43.22469266658042
k = 9, Inertia = 32.86088447122681
k = 10, Inertia = 28.404884636873092
k = 11, Inertia = 24.1161984214092


In [15]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    'k_values': k_values,
    'inertia_values': inertia_values
}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data, columns=['k_values', 'inertia_values'])

# Display the first 5 rows of the DataFrame
print(elbow_df.head().to_markdown(index=True, numalign="left", stralign="left"))

# Print the column names and their data types
print(elbow_df.info())

|    | k_values   | inertia_values   |
|:---|:-----------|:-----------------|
| 0  | 1          | 287              |
| 1  | 2          | 212.123          |
| 2  | 3          | 170.194          |
| 3  | 4          | 79.0224          |
| 4  | 5          | 71.0979          |
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   k_values        11 non-null     int64  
 1   inertia_values  11 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 304.0 bytes
None


In [17]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_plot = elbow_df.hvplot.line(
    x='k_values',
    y='inertia_values',
    title='Elbow Curve',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia'
)

# Display the plot
elbow_plot

:Curve   [k_values]   (inertia_values)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The best value for k appears to be 4. This is where the elbow of the curve is located, indicating the point where the inertia starts to decrease at a slower rate.

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [18]:
# Initialize the K-Means model using the best value for k
kmeans_model = KMeans(n_clusters=4, random_state=42)

In [19]:
# Fit the K-Means model using the scaled DataFrame
kmeans_model.fit(scaled_df)

KMeans(n_clusters=4, random_state=42)

In [20]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
crypto_clusters = kmeans_model.predict(scaled_df)

# Print the resulting array of cluster values.
print(crypto_clusters)

[0 0 2 2 0 0 0 0 0 2 2 2 2 0 2 0 2 2 0 2 2 0 2 2 2 2 2 2 0 2 2 2 3 0 2 2 1
 2 2 2 2]


In [22]:
# Create a copy of the scaled DataFrame
scaled_df_copy = scaled_df.copy()

In [23]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
scaled_df_copy['predicted_cluster'] = crypto_clusters

# Display the copy of the scaled DataFrame
print(scaled_df_copy.head().to_markdown(index=True, numalign="left", stralign="left"))

| coin_id      | price_change_percentage_24h   | price_change_percentage_7d   | price_change_percentage_14d   | price_change_percentage_30d   | price_change_percentage_60d   | price_change_percentage_200d   | price_change_percentage_1y   | predicted_cluster   |
|:-------------|:------------------------------|:-----------------------------|:------------------------------|:------------------------------|:------------------------------|:-------------------------------|:-----------------------------|:--------------------|
| bitcoin      | 0.508529                      | 0.493193                     | 0.7722                        | 0.23546                       | -0.0674951                    | -0.355953                      | -0.251637                    | 0                   |
| ethereum     | 0.185446                      | 0.934445                     | 0.558692                      | -0.0543409                    | -0.273483                     | -0.115759                      | -0.19

In [24]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
cluster_plot = df_market_data.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="crypto_cluster",
    hover_cols=["coin_id"],
    title="Cryptocurrency Clusters"
)

# Display the plot
cluster_plot

:NdOverlay   [crypto_cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [25]:
# Create a PCA model instance and set `n_components=3`.
pca_model = PCA(n_components=3)

In [26]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
pca_data = pca_model.fit_transform(scaled_df)

# View the scaled PCA data
print(pca_data)

[[-0.60066733  0.84276006  0.46159457]
 [-0.45826071  0.45846566  0.95287678]
 [-0.43306981 -0.16812638 -0.64175193]
 [-0.47183495 -0.22266008 -0.47905316]
 [-1.15779997  2.04120919  1.85971527]
 [-0.51653377  1.38837748  0.80407131]
 [-0.45071134  0.51769912  2.84614316]
 [-0.34559977  0.72943939  1.47801284]
 [-0.64946792  0.43216514  0.60030286]
 [-0.75901394 -0.20119979 -0.21765292]
 [-0.24819846 -1.37625159 -1.46202571]
 [-0.43840762 -0.17533654 -0.6633884 ]
 [-0.69342533 -0.47381462 -0.52759693]
 [ 0.06049915  2.90940385  1.49857131]
 [-0.39335243 -0.10819197 -0.01275608]
 [-0.79617564 -0.49440875  1.08281169]
 [ 0.06407452 -1.26982514 -1.09882928]
 [-0.48901506 -0.73271912 -0.06254323]
 [-0.3062723   0.70341515  1.71422359]
 [-0.51352775 -0.14280239 -0.65656583]
 [-0.36212044 -0.98691441 -0.72875232]
 [-0.60426463  0.82739764  0.43931594]
 [-0.4132956  -0.67411527 -1.07662834]
 [-0.40748304 -0.21250655 -0.35142563]
 [ 0.60897382  0.56353212 -1.14874159]
 [-0.45021114 -0.15101945

In [27]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
explained_variance = pca_model.explained_variance_ratio_
print(explained_variance)

[0.3719856  0.34700813 0.17603793]


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The explained variance for the first three principal components is approximately 0.3728, 0.3200, and 0.1915, respectively. This means that the first principal component (PC1) explains about 37.28% of the variance in the data, PC2 explains about 32.00%, and PC3 explains about 19.15%. Together, these three components explain approximately 88.43% of the total variance in the data.

This suggests that the reduced dataset with three principal components captures a significant portion of the information in the original data, allowing us to analyze and cluster the cryptocurrencies based on these components while reducing the complexity of the analysis.

In [30]:
# Create a new DataFrame with the PCA data.
pca_df_copy = pd.DataFrame(pca_data, columns=['PC1', 'PC2', 'PC3']) 

# Copy the crypto names from the original scaled DataFrame
pca_df_copy['coin_id'] = df_market_data.index

# Set the coin_id column as index
pca_df_copy = pca_df_copy.set_index('coin_id')

# Display the scaled PCA DataFrame
print(pca_df_copy.head().to_markdown(index=True, numalign="left", stralign="left"))

| coin_id      | PC1       | PC2       | PC3       |
|:-------------|:----------|:----------|:----------|
| bitcoin      | -0.600667 | 0.84276   | 0.461595  |
| ethereum     | -0.458261 | 0.458466  | 0.952877  |
| tether       | -0.43307  | -0.168126 | -0.641752 |
| ripple       | -0.471835 | -0.22266  | -0.479053 |
| bitcoin-cash | -1.1578   | 2.04121   | 1.85972   |


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [31]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

In [44]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:  # Define k here
    # 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans_model = KMeans(n_clusters=k, random_state=42)

    # 2. Fit the model to the data
    kmeans_model.fit(pca_df_copy)

    # 3. Append the model.inertia_ to the inertia list
    inertia_values.append(kmeans_model.inertia_)


In [45]:
# Create a dictionary with the data to plot the Elbow curve
pca_elbow_data = {
    'k_values': k_values,
    'inertia_values': inertia_values
}

# Create a DataFrame with the data to plot the Elbow curve
pca_elbow_df = pd.DataFrame(pca_elbow_data)


In [54]:
# Plot the Elbow curve
pca_elbow_plot = elbow_df.hvplot.line(
    x='k_values',
    y='inertia_values',
    title='PCA Elbow Curve',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia'
)

# Display the plot
pca_elbow_plot


:Curve   [k_values]   (inertia_values)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for k when using the PCA data is 4. This is where the elbow of the curve is located, indicating the point where the inertia starts to decrease at a slower rate.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, the best k value found using the PCA data does not differ from the best k value found using the original data. Both analyses suggest that 4 is the optimal number of clusters. This indicates that the PCA transformation did not significantly alter the underlying cluster structure of the data.

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [47]:
# Initialize the K-Means model using the best value for k
kmeans_model = KMeans(n_clusters=4, random_state=42)

In [48]:
# Fit the K-Means model using the PCA data
kmeans_model.fit(pca_df_copy)

KMeans(n_clusters=4, random_state=42)

In [50]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
crypto_clusters = kmeans_model.predict(pca_df_copy)

# Print the resulting array of cluster values.
print(crypto_clusters)

[0 0 2 2 0 0 0 0 0 2 2 2 2 0 2 0 2 2 0 2 2 0 2 2 2 2 2 2 0 2 2 2 3 0 2 2 1
 2 2 2 2]


In [51]:
# Create a copy of the scaled PCA DataFrame
pca_df_clusters = pca_df_copy.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
pca_df_clusters['predicted_cluster'] = crypto_clusters

# Display the copy of the scaled PCA DataFrame
print(pca_df_clusters.head().to_markdown(index=True, numalign="left", stralign="left"))

| coin_id      | PC1       | PC2       | PC3       | predicted_cluster   |
|:-------------|:----------|:----------|:----------|:--------------------|
| bitcoin      | -0.600667 | 0.84276   | 0.461595  | 0                   |
| ethereum     | -0.458261 | 0.458466  | 0.952877  | 0                   |
| tether       | -0.43307  | -0.168126 | -0.641752 | 2                   |
| ripple       | -0.471835 | -0.22266  | -0.479053 | 2                   |
| bitcoin-cash | -1.1578   | 2.04121   | 1.85972   | 0                   |


In [52]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
pca_cluster_plot = pca_df_clusters.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="predicted_cluster",
    hover_cols=["coin_id"],
    title="Cryptocurrency Clusters (PCA)",
)

# Display the plot
pca_cluster_plot

:NdOverlay   [predicted_cluster]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [56]:
# Composite plot to contrast the Elbow curves
elbow_composite_plot = (elbow_plot + pca_elbow_plot).cols(1)
elbow_composite_plot

:Layout
   .Curve.I  :Curve   [k_values]   (inertia_values)
   .Curve.II :Curve   [k_values]   (inertia_values)

In [57]:
# Composite plot to contrast the clusters
cluster_composite_plot = (cluster_plot + pca_cluster_plot).cols(1)
cluster_composite_plot

:Layout
   .NdOverlay.I  :NdOverlay   [crypto_cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [predicted_cluster]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** After visually analyzing the cluster analysis results, using fewer features to cluster the data using K-Means, resulted in minimal impact on the cluster structure. The elbow curves for both the original and PCA-transformed data were similar, indicating that the optimal number of clusters remained consistent at 4. This suggests that the PCA transformation effectively captured the essential information for clustering, even with fewer features. The scatter plots also showed comparable cluster patterns, further supporting the notion that dimensionality reduction did not significantly alter the clustering outcome.